# WSN-Localization
Localization algorithms are used to locate the source of a signal, based on the characteristics of the signal when it is received at different locations. The process involves transmitting a signal from a device at an unknown location (called the “target” node) and recording signal characteristics at receiver nodes at known locations, also called “anchor” nodes. Some applications include locating the center of the rotor core in hearts with Atrial Fibrillation (AFib), which allows for treatment, and locating the source of a forest fire. Several localization techniques exist, such as Time of Arrival (TOA), Time Difference of Arrival (TDOA), and Modified Time Difference of Arrival (mTDOA). It has been shown that mTDOA is particularly useful in the case of AFib modeled by the FitzHugh-Nagumo (FN) model since it gives the signal propagation speed as an output rather than an input. In mTDOA, using the locations of the anchor nodes and the relative times of reception of the signal by the anchor nodes, the location of the target node, the propagation speed, and other auxiliary information are estimated. 

This project aims to improve this algorithm by considering the entire received signal, rather than only the initial reception time, and applying synchronization-based techniques to estimate time delays. In synchronization, the influence delay between two signals can be deduced using the mutual information (MI) between time-shifted versions of those signals. When the MI is at a maximum, it is likely that the particular time delay that yields the maximum MI is the correct estimate of the delay between the transmitted and received signal. Outputs from the FN model at the locations of the anchor nodes can be put under this MI Time Shift process to learn the time difference between two anchor nodes, which can be fed to the standard mTDOA algorithm. 

## The Theory
There are two major kinds of signal localization, localization involving a single transmitted “message” or signal, and that involving a continuous signal received over time. Both use many “anchor nodes”, which receive the signal and one or more target nodes which transmit the signal.

### Single Message
With the single message system, it is easy to determine the time delay of the received signal between two anchor nodes; the received time at one node can be subtracted from the received time at another node. These time delays, denoted $t_{jk}$ for the time difference between the $j$ th and $k$ th nodes, can be fed into matrices which can produce an estimate for the location of the target node. In each localization method, there are three matrices/vectors, coming together into the equation, $\textbf{Hx}=\textbf{b}$. $\textbf{x}$ contains the location the target node and sometimes other helpful information, such as the time of transmission or the speed of propagation. $\textbf{H}$ and $\textbf{b}$ contain the gathered information from the time delays. $r_j$ is the position of the $j$ th node (with 0 being target), $t_j$ is the absolute time of arrival of the message at the $j$ th node, and $c$ is the propagation speed. These are the matrices for mTDOA. 

$
\textbf{H}=\
\begin{bmatrix}\
2(\textbf{r}_2 - \textbf{r}_1) & -2(t_2-t_1) & (t_2^2-t_1^2)\\\
2(\textbf{r}_3 - \textbf{r}_1) & -2(t_3-t_1) & (t_3^2-t_1^2)\\\
2(\textbf{r}_4 - \textbf{r}_1) & -2(t_4-t_1) & (t_4^2-t_1^2)\\\
\vdots\\\
2(\textbf{r}_n - \textbf{r}_1) & -2(t_n-t_1) & (t_n^2-t_1^2)\
\end{bmatrix}\
\\\
\\\
b=\
\begin{bmatrix}\
||\textbf{r}_2||^2 - ||\textbf{r}_1||^2\\\
||\textbf{r}_3||^2 - ||\textbf{r}_1||^2\\\
||\textbf{r}_4||^2 - ||\textbf{r}_1||^2\\\
\vdots\\\
||\textbf{r}_N||^2 - ||\textbf{r}_1||^2\
\end{bmatrix}\
\\\
\\\
\textbf{x}=\
\begin{bmatrix}\
\textbf{r}_0\\\
c^2 t_0\\\
c^2\
\end{bmatrix}\
$

The code supports TOA, TDOA, mTDOA, and cTDOA. 

If the system includes multiple target nodes, an iterative approach may be needed to localize all of them, even transmitting from the same target node twice. 

### Continuous Signal
In many cases, the received signal is not one event but a continuous, time-varying signal. The time delay between two signals can be determined using the Mutual Information (MI) Time Shift method mentioned in the introduction. For each time shift, the MI between the two signals is calculated, and the time shift that yields the highest MI is said to be the time delay between the two received signals. Once the time delays between the received signals at each node and the received signal at node 1 (the “base” anchor) are calculated, the time delays can be fed into whichever algorithm is needed. Typically, mTDOA or cTDOA will be used for this because they do not require the propagation speed. 

The code supports a few signal types including Gaussian Auto-Regressive (`"ar"`) and FitzHugh-Nagumo (`"fn"`). 

## The Code
### Initializing a WSN
The localization algorithms are implemented in the `WSN` class (`WSN.py`). To run the algorithms, a `WSN` can be created and worked with directly, or an `App` (`App.py`) can be created which creates a `WSN` for itself. Assuming, a `WSN` is being used directly, it accepts the parameters: `N`, the number of nodes (anchors + targets); and `size`, the width of the location space. The location space is always square, $[0, size)^2$. 100 is typically used for `size`. 

### Creating the nodes
Then, the nodes must be created by calling either `reset_nodes` or `reset_clusters`. `reset_nodes` randomizes `N` node positions and stores them in `self.nodes`. If cTDOA is being used, `reset_clusters` should be called instead so that the nodes are initialized in clusters; and `num_clusters` and `cluster_size` should either be set by passing key word arguments to `WSN`’s initializer or setting them before calling `reset_clusters`. `reset_clusters` chooses `num_clusters` random positions around the location space and then places `cluster_size` nodes around each cluster. It also resets `self.N` to `num_clusters` * `cluster_size`. 

Alternatively, the nodes can be set manually be assigning the `WSN`’s variable, `nodes`, with a numpy array of shape `(N, 2)`. 

### Selecting anchor nodes
After the nodes are created, the anchor nodes should be selected by passing a set of indices to `reset_anchors`. One can select the first `n` nodes to be anchor nodes by calling `reset_anchors(set(range(n)))`. If cTDOA is being used, then all nodes should be anchor nodes. (The estimated position of the target node should be the center of the rotor core in the FN model.)
